In [1]:
import pandas as pd
import numpy as np
import anndata

from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt

import scanpy as sc

import networkx as nx

In [2]:
from genexpression_utils import fetch_genexpression_data, get_marker_gene_sets, fetch_fibroblast_data, fetch_myotube_data, fetch_HYB_genexpression, fetch_hyb_nodes

In [3]:
from mapping import get_sorted_gene_order, generate_gene_id_name_map, get_master_regulator_list, get_TF_lists, get_a_matrix_threshold

from network_utils import GeneUtils

a_matrix_adata, b_matrix_true = get_a_matrix_threshold(300)
gene_id_name_map, gene_name_id_map = generate_gene_id_name_map()
GUtils = GeneUtils

 Total Activators : 174
Total Reprossors : 705
Total Conflicted : 729
Total Transcription Factors : 1608


ValueError: too many values to unpack (expected 2)

In [ ]:
print(HYB_anndata)
print(HYB_anndata.layers)

In [ ]:
HYB_anndata = fetch_HYB_genexpression()

fibroblast = HYB_anndata[HYB_anndata.obs_names == 'Control']
reprogrammed = HYB_anndata[HYB_anndata.obs_names == 'mmMYOD1']
myotube = fetch_myotube_data()

In [ ]:
marker_gene_sets = get_marker_gene_sets()
myotube_markers = set(marker_gene_sets['Muscle'].tolist())

muscle_marker_genes = list(myotube_markers)


fib_markers = set(marker_gene_sets['Fibroblast'].tolist())

fib_marker_genes = list(fib_markers)

fib_marker_ids = [ ]
for i in fib_marker_genes:
    try:
        gid = gene_name_id_map[i]
        fib_marker_ids.append(gid)
    except:
        print(f'no mapping found for {i}')
        continue

muscle_marker_ids = [ ]
for i in fib_marker_genes:
    try:
        gid = gene_name_id_map[i]
        muscle_marker_ids.append(gid)
    except:
        print(f'no mapping found for {i}')
        continue
    

In [10]:
def process_gex_anndata(fibroblast, fib_marker_ids, thresh):

    sc.pp.normalize_total(fibroblast, target_sum=1e4)
    sc.pp.log1p(fibroblast)
    
    fibroblast.var['GeneName'] = fibroblast.var_names
    fibroblast.var_names = fibroblast.var['gene_id']

    gex_adata = fibroblast

    
    
    expr_values = gex_adata.X.flatten()
    
    expr_series = pd.Series(expr_values, index=gex_adata.var_names)

    if fib_marker_ids:
        marker_expr = expr_series[expr_series.index.isin(fib_marker_ids)]

    if thresh:
        threshold = thresh
    else:
        threshold = marker_expr.mean()
    
    print(threshold)
    high_expr = expr_series[expr_series > threshold]
    top_genes = high_expr.sort_values(ascending=False)
    top_genes = top_genes[top_genes.index.notna()]
    fibroblast_expressed_genes = top_genes.index.tolist()
    
    fibroblast_adata = a_matrix_adata[a_matrix_adata.obs_names.isin(fibroblast_expressed_genes), a_matrix_adata.var_names.isin(fibroblast_expressed_genes)].copy()
    
    return fibroblast_adata

0.38259256


/home/nuvi/.local/lib/python3.11/site-packages/scanpy/preprocessing/_normalization.py:216: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [12]:
thresh = 0.38259256

In [13]:
hyb_fib = process_gex_anndata(fibroblast, fib_marker_ids, None)
hyb_reprogrammed = process_gex_anndata(reprogrammed, None, thresh)

/home/nuvi/.local/lib/python3.11/site-packages/scanpy/preprocessing/_normalization.py:216: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


0.38259256


In [14]:
hyb_reprogrammed

AnnData object with n_obs × n_vars = 3622 × 3622
    layers: 'HURI', 'STRING', 'HWG_0', 'HWG_300'

In [15]:
hyb_fib

AnnData object with n_obs × n_vars = 2782 × 2782
    layers: 'HURI', 'STRING', 'HWG_0', 'HWG_300'

In [16]:
myotube

AnnData object with n_obs × n_vars = 4046 × 4046
    layers: 'HURI', 'STRING', 'HWG_0', 'HWG_300'